## 05 Merge Interim Data of all Players (FBREF and Transfermarkt)

In [ ]:
# TODO: fix merge issues; transfermarkt data is missing for all players after merge

In [11]:
import sys
sys.path.append("..")
from src.preprocessing.merge_player_data import run_merge_pipeline

df_all_players = run_merge_pipeline("Valencia CF", ["2223", "2324", "2425"])

print(df_all_players.shape)
df_all_players.head()

(120, 179)


,Player,Nation,Pos,Age_x,MP,Starts,Min,90s,Gls,Ast,...,CPA,Mis,Dis,Rec,PrgR_player_possession,MarketValueEuro,Age_y,Position,Nationality,Current club
0,Giorgi Mamardashvili,ge GEO,GK,21.0,38,38,3420.0,38.0,0.0,0.0,...,0.0,2.0,1.0,470.0,0.0,NaN,NaN,NaN,NaN,NaN
1,Samuel Lino,br BRA,"FW,MF",22.0,38,33,2887.0,32.1,6.0,1.0,...,60.0,85.0,57.0,1165.0,260.0,NaN,NaN,NaN,NaN,NaN
2,José Luis Gayà,es ESP,DF,27.0,31,31,2709.0,30.1,1.0,3.0,...,8.0,19.0,12.0,1132.0,200.0,NaN,NaN,NaN,NaN,NaN
3,Domingos André Ribeiro Almeida,pt POR,MF,22.0,34,29,2539.0,28.2,2.0,4.0,...,3.0,42.0,49.0,1114.0,71.0,NaN,NaN,NaN,NaN,NaN
4,Mouctar Diakhaby,gn GUI,DF,25.0,29,28,2446.0,27.2,3.0,0.0,...,1.0,14.0,3.0,868.0,11.0,NaN,NaN,NaN,NaN,NaN


In [12]:
df_all_players[df_all_players['NormalizedName'] == 'javier guerra']

,Player,Nation,Pos,Age_x,MP,Starts,Min,90s,Gls,Ast,...,CPA,Mis,Dis,Rec,PrgR_player_possession,MarketValueEuro,Age_y,Position,Nationality,Current club
19,Javier Guerra,es ESP,MF,19.0,10,6,605.0,6.7,1.0,0.0,...,5.0,8.0,20.0,218.0,16.0,NaN,NaN,NaN,NaN,NaN
47,Javier Guerra,es ESP,MF,20.0,36,28,2498.0,27.8,4.0,1.0,...,9.0,50.0,52.0,833.0,68.0,NaN,NaN,NaN,NaN,NaN
82,Javier Guerra,es ESP,MF,21.0,36,31,2581.0,28.7,3.0,3.0,...,13.0,50.0,54.0,1136.0,84.0,NaN,NaN,NaN,NaN,NaN
